In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Carga inical

Link dataset kaggle (https://www.kaggle.com/datasets/whenamancodes/student-performance)

In [ ]:
dataset_original = pd.read_csv("./archivos/Maths.csv")
dataset_original